In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import warnings
warnings.filterwarnings('ignore')

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
home_train=pd.read_csv("../input/application_train.csv")
home_test=pd.read_csv("../input/application_test.csv")
bureau = pd.read_csv('../input/bureau.csv')
bureau_balance = pd.read_csv('../input/bureau_balance.csv')
bureau=pd.get_dummies(bureau)
bureau_balance=pd.get_dummies(bureau_balance)
bureau_balance_agg = bureau_balance.groupby('SK_ID_BUREAU', as_index = False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
columns=['SK_ID_BUREAU']
for var in bureau_balance_agg.columns.levels[0]:
    if var != 'SK_ID_BUREAU':
        for stat in bureau_balance_agg.columns.levels[1][:-1]:
            columns.append('bureau_balance_%s_%s' % (var, stat))
bureau_balance_agg.columns = columns
bureau=bureau.merge(bureau_balance_agg,on = 'SK_ID_BUREAU',how='left')
loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'loan_counts'})
home_train = home_train.merge(loan_counts, on = 'SK_ID_CURR', how = 'left')
home_train['loan_counts'] = home_train['loan_counts'].fillna(0)
bureau_agg = bureau.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR', as_index = False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

columns = ['SK_ID_CURR']
for var in bureau_agg.columns.levels[0]:
    if var != 'SK_ID_CURR':
        for stat in bureau_agg.columns.levels[1][:-1]:
            columns.append('bureau_%s_%s' % (var, stat))
bureau_agg.columns = columns
home_train = home_train.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
home_test = home_test.merge(loan_counts, on = 'SK_ID_CURR', how = 'left')
home_test['loan_counts'] = home_test['loan_counts'].fillna(0)
home_test = home_test.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')

In [ ]:
one_hot_encoded_home_train = pd.get_dummies(home_train)
one_hot_encoded_home_test = pd.get_dummies(home_test)
one_hot_encoded_home_train.head()
y=one_hot_encoded_home_train['TARGET']
final_train, final_test = one_hot_encoded_home_train.align(one_hot_encoded_home_test, join='inner', axis=1)
my_imputer = SimpleImputer()
imputed_home_train = pd.DataFrame(my_imputer.fit_transform(final_train))
imputed_home_test = pd.DataFrame(my_imputer.transform(final_test))
imputed_home_train.columns = final_train.columns
imputed_home_test.columns = final_test.columns

from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(imputed_home_train, y, test_size=0.25)

model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
model.fit(train_X, train_y, early_stopping_rounds=5, eval_set=[(test_X, test_y)], verbose=False)

predictions=model.predict(imputed_home_test)
submit = home_test[['SK_ID_CURR']]
submit['TARGET'] = abs(predictions)


submit.to_csv('xgbp.csv', index = False)